In [ ]:
!pip3 install git+https://github.com/holounic/Bayesian-Optimization.git
!pip3 install cma

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/holounic/Bayesian-Optimization.git to /tmp/pip-req-build-l1s05xv9
  Running command git clone -q https://github.com/holounic/Bayesian-Optimization.git /tmp/pip-req-build-l1s05xv9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/holounic/pyDOE to /tmp/pip-install-p2hlvr5j/pydoe_4f229d2770184431b2ee067a18f18d19
  Running command git clone -q https://github.com/holounic/pyDOE /tmp/pip-install-p2hlvr5j/pydoe_4f229d2770184431b2ee067a18f18d19
     |████████████████████████████████| 316 kB 5.2 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Created wheel for bayes-optim: filename=bayes_optim-0.2.7-py3-none-any.whl size=145056 sha256=08af904f5e075496a0f7093d62207ee777d048bf11aa095d4a706039979fb2db
  Stored in directory: /tmp/pip-ephem-wheel-cache-6

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import cma

from pyDOE import lhs

from bayes_optim import BO
from bayes_optim.search_space import RealSpace
from bayes_optim.surrogate import GaussianProcess, trend

from benchmark.bbobbenchmarks import F13

from tqdm.notebook import trange

# Constants

In [ ]:
np.random.seed(1111)

dim = 2
lb, ub = -5, 5

mean = trend.constant_trend(dim, beta=None)
thetaL = 1e-10 * (ub - lb) * np.ones(dim)
thetaU = 10 * (ub - lb) * np.ones(dim)
theta0 = np.random.rand(dim) * (thetaU - thetaL) + thetaL

max_FEs = 2**5
verbose = False

doe_budget = 0.2
doe_size = int(doe_budget * max_FEs)

n_samples = 30000

max_cma_iter = 20

f = F13

test_seeds = list(range(50))

space = RealSpace([lb, ub]) * dim

pop_size = int(4 + 3 * np.log(doe_size * dim) // 1)

n_tests = 3

f_name = 'f13'

# Make optimizer function

In [ ]:
def make_optimizer(obj_func, doe):
    model = GaussianProcess(
        mean=mean,
        corr="squared_exponential",
        theta0=theta0,
        thetaL=thetaL,
        thetaU=thetaU,
        nugget=0,
        noise_estim=False,
        optimizer="BFGS",
        wait_iter=3,
        random_start=dim,
        likelihood="concentrated",
        eval_budget=100 * dim
    )
    return BO(
        search_space=space,
        obj_fun=obj_func,
        model=model,
        DoE_size=len(doe),
        acquisition_fun="EI",
        initial_points=doe,
        max_FEs=max_FEs,
        verbose=verbose,
        n_point=1
    )

# Experiment code

In [ ]:
def regret(obj_func, doe):
  bo = make_optimizer(obj_func, doe)
  xopt, yopt, _ = bo.run()
  return yopt - obj_func.getfopt()

def fitness_function(doe_size, n_individuals, func_seeds):
  i = 0
  def func(X):
    nonlocal i
    X = X.clip(lb, ub, out=X)
    doe = np.split(X, doe_size)
    obj_func = None
    k = i // n_individuals
    func_seed_set = func_seeds[k]
    i += 1
    r = 0
    print(func_seed_set)
    for func_seed in func_seed_set:
      r += regret(f(func_seed), doe)
    return r / len(func_seed_set)
  return func

def cma_es_experiment(doe_size, func_seeds, individual0, sigma0 = ub / 3):
  # run cma-es 
  doe_flattened, es = cma.fmin2(fitness_function(doe_size, pop_size, func_seeds), individual0, sigma0, {'maxiter': max_cma_iter, 'popsize': pop_size})
  
  # split a vector into set of points
  doe = np.split(doe_flattened, doe_size)

  return doe, es

# Testing code

In [ ]:
def additional_metrics(history):
  for h in history:
    func = f(h['f_s'])
    y = func(h['xopt'])
    real_x = func._getxopt()
    d = np.linalg.norm(h['doe'] - real_x, axis=1)
    h['min_dist_to_opt'] = d.min()
    h['max_dist_to_opt'] = d.max()
    h['real_xopt'] = real_x
    h['real_yopt'] = func.getfopt()
    h['bo_history'] = h['bo'].history
  return history

In [ ]:
def compute_all(func_seed, doe, n=3, n_points=1000):
  regrets, history, precisions = [], [], []
  for _ in range(n):
    obj_func = f(func_seed)
    bo = make_optimizer(obj_func, doe)
    xopt, yopt, _ = bo.run()
    regret = yopt - obj_func.getfopt()
    points = lhs(dim, n_points) * 10 - 5
    predicted_y = np.concatenate(bo.model.predict(points))
    real_y = obj_func(points)
    precision = np.mean((predicted_y - real_y)**2)
    precisions.append(precision)
    regrets.append(regret)
    history.append({'xopt': xopt, 'yopt': yopt, 'regret': regret, 'doe': doe, 'precision': precision, 'bo': bo, 'f_s': func_seed})
  return np.concatenate(regrets).mean(), np.mean(precisions), history

def test(doe, n=10):
  precisions, regrets, history = [], [], []
  for seed in trange(n):
    doe = doe
    r, p, h = compute_all(seed, doe, n=1, n_points=1000)
    precisions.append(p)
    regrets.append(r)
    history.append(h[0])
  return precisions, regrets, history

# Experiments

In [ ]:
individual0 = np.concatenate(lhs(2, doe_size) * 10 - 5)
individual0

array([-1.54901728,  1.34212926,  3.81413919, -3.47714037,  2.30841543,
        2.00327005,  0.50808908,  4.5586762 , -4.72791635, -2.09921999,
       -1.67488531, -1.05830264])

In [ ]:
func_seeds = np.random.randint(0, 2**31, (max_cma_iter, n_tests)).tolist()
func_seeds.append(test_seeds)
func_seeds

[[1475636620, 1134109346, 1333488308],
 [1005703320, 8632214, 651166484],
 [1011871627, 586793230, 1021307656],
 [1790440934, 1013250956, 23639534],
 [2128019186, 1941953366, 1221597576],
 [441380969, 543340478, 721493309],
 [458055347, 1529544234, 2001974348],
 [985630174, 1018474702, 2025616588],
 [1868994436, 1247560269, 1046561192],
 [104666121, 1648579849, 1890662871],
 [1453389842, 1597818624, 666513316],
 [104385800, 637573627, 705513029],
 [597623957, 101841489, 590019245],
 [1874809778, 476201470, 1524655072],
 [1846915629, 319501579, 1359356797],
 [114995871, 1157171255, 1425728285],
 [366794197, 1281455660, 708681325],
 [1428655094, 909226738, 2134097890],
 [1663587672, 1748947072, 1151088343],
 [1358540125, 1044006575, 1473510937],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42

In [ ]:
doe, es = cma_es_experiment(doe_size, func_seeds, individual0, sigma0 = ub / 3)

(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 12 (seed=695013, Wed Nov 16 15:44:04 2022)
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
[1475636620, 1134109346, 1333488308]
[1475636620, 1134109346, 1333488308]
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308]
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     11 1.195354358793006e+01 1.0e+00 1.53e+00  

In [ ]:
doe

[array([-0.60934463, -0.53824901]),
 array([ 3.57933552, -3.17003053]),
 array([-1.18097613,  4.98047291]),
 array([-2.17753949,  5.        ]),
 array([-4.10475949, -0.25245231]),
 array([-0.71669831, -1.93372999])]

In [ ]:
precision, regrets, history = test(doe, n=50)

  0%|          | 0/50 [00:00<?, ?it/s]

Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...


In [ ]:
np.mean(precision)

1057220.6893083798

In [ ]:
pd.DataFrame(additional_metrics(history)).drop('bo', axis=1).to_csv(f'{f_name}_cma_es_doe.csv')

In [ ]:
doe_lhs = np.split(individual0, doe_size)
doe_lhs

[array([-1.54901728,  1.34212926]),
 array([ 3.81413919, -3.47714037]),
 array([2.30841543, 2.00327005]),
 array([0.50808908, 4.5586762 ]),
 array([-4.72791635, -2.09921999]),
 array([-1.67488531, -1.05830264])]

In [ ]:
lhs_precision, lhs_regrets, lhs_history = test(doe_lhs, n=50)

  0%|          | 0/50 [00:00<?, ?it/s]

Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...


In [ ]:
np.mean(lhs_precision)

1058052.3795259842

In [ ]:
pd.DataFrame(additional_metrics(lhs_history)).drop('bo', axis=1).to_csv(f'{f_name}_lhs_initial_doe.csv')